Chargement des zip

In [0]:
import requests
import zipfile
from azure.storage.blob import BlobServiceClient
from io import BytesIO
import os

DefaultEndpointsProtocol = os.environ["DefaultEndpointsProtocol"]
AccountName = os.environ["AccountName"]
AccountKey = os.environ["AccountKey"]
EndpointSuffix = os.environ["EndpointSuffix"]

# 1. Définir les variables
urls = ["https://www.data.gouv.fr/api/1/datasets/r/6994a9f1-3f4b-4e15-a4dc-0e358a6aac13","https://www.data.gouv.fr/api/1/datasets/r/c0350599-a041-4724-9942-ad4c2ba9a7b3","https://www.data.gouv.fr/api/1/datasets/r/96452cf0-329a-4908-8adb-8f061adcca4c","https://www.data.gouv.fr/api/1/datasets/r/77d3151a-739e-4aab-8c34-7a15d7fea55d","https://www.data.gouv.fr/api/1/datasets/r/3c5ebbd9-f6b5-4837-a194-12bfeda7f38e"]

blob_connecting_string = f"DefaultEndpointsProtocol={DefaultEndpointsProtocol};AccountName={AccountName};AccountKey={AccountKey};EndpointSuffix={EndpointSuffix}"

container_name = "lnd-fichiers-source"

# 2. Envoyer dans ton Blob
blob_service_client = BlobServiceClient.from_connection_string(blob_connecting_string)

container_client = blob_service_client.get_container_client(container_name)

for annee, url in zip(["2025","2024","2023","2022","2021"],urls):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Erreur lors du téléchargement du fichier : {response.status_code}")
    zip_bytes = BytesIO(response.content)
    zip_name = f"dis-{annee}-dept.zip"
    container_client.upload_blob(name=zip_name, data=zip_bytes, overwrite=True)

    print(f"{zip_name} envoyé dans le blob ✅")


Dézippage dans Azure

In [0]:
import zipfile

# Connexion à Azure
blob_service_client = BlobServiceClient.from_connection_string(blob_connecting_string)
source_container = blob_service_client.get_container_client("lnd-fichiers-source")
dest_container_name = "lnd-fichiers-dezip"

# Créer le conteneur de destination si nécessaire
if not blob_service_client.get_container_client(dest_container_name).exists():
    blob_service_client.create_container(dest_container_name)

dest_container = blob_service_client.get_container_client(dest_container_name)

# Lister les blobs ZIP
blobs_list = source_container.list_blobs()
for blob in blobs_list:
    if blob.name.endswith(".zip"):
        print(f"Décompression de {blob.name}...")

        # Télécharger le ZIP en mémoire
        blob_data = source_container.download_blob(blob.name).readall()
        zip_bytes = BytesIO(blob_data)

        # Dézipper
        with zipfile.ZipFile(zip_bytes) as z:
            for file_name in z.namelist():
                # Lire le fichier dans la mémoire
                file_data = z.read(file_name)

                # Envoyer le fichier dans le blob de destination
                dest_blob_name = file_name  # tu peux rajouter un préfixe si tu veux
                dest_container.upload_blob(name=dest_blob_name, data=file_data, overwrite=True)
                print(f"  - {file_name} envoyé dans {dest_container_name} ✅")
